In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

# Load the preprocessed data into a pandas dataframe
df = pd.read_csv('motor_current_readings.csv')

# Extract features from the current readings
df['rms_current'] = np.sqrt(df['phase1_current']**2 + df['phase2_current']**2 + df['phase3_current']**2)
df['power_factor'] = (df['phase1_current']*df['phase1_voltage'] + df['phase2_current']*df['phase2_voltage'] + df['phase3_current']*df['phase3_voltage'])/(df['rms_current']*df['total_power'])
df['harmonic_content'] = abs(np.fft.fft(df['rms_current']))
df['crest_factor'] = df[['phase1_current', 'phase2_current', 'phase3_current']].abs().max(axis=1)/df['rms_current']

# Create condition indicators from the features
df['mean_rms_current'] = df['rms_current'].rolling(window=10).mean()
df['std_deviation_rms_current'] = df['rms_current'].rolling(window=10).std()
df['skewness_rms_current'] = df['rms_current'].rolling(window=10).skew()

# Train an Isolation Forest model on the condition indicators
X = df[['mean_rms_current', 'std_deviation_rms_current', 'skewness_rms_current']]
model = IsolationForest(n_estimators=100, contamination=0.01)
model.fit(X)

# Predict anomalies on new data
new_data = pd.read_csv('new_motor_current_readings.csv')
new_data['rms_current'] = np.sqrt(new_data['phase1_current']**2 + new_data['phase2_current']**2 + new_data['phase3_current']**2)
new_data['power_factor'] = (new_data['phase1_current']*new_data['phase1_voltage'] + new_data['phase2_current']*new_data['phase2_voltage'] + new_data['phase3_current']*new_data['phase3_voltage'])/(new_data['rms_current']*new_data['total_power'])
new_data['harmonic_content'] = abs(np.fft.fft(new_data['rms_current']))
new_data['crest_factor'] = new_data[['phase1_current', 'phase2_current', 'phase3_current']].abs().max(axis=1)/new_data['rms_current']

new_data['mean_rms_current'] = new_data['rms_current'].rolling(window=10).mean()
new_data['std_deviation_rms_current'] = new_data['rms_current'].rolling(window=10).std()
new_data['skewness_rms_current'] = new_data['rms_current'].rolling(window=10).skew()

y_pred = model.predict(new_data[['mean_rms_current', 'std_deviation_rms_current', 'skewness_rms_current']])


KeyError: 'phase1_current'